In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px  # For Plotly Express, which is easy-to-use for quick visualizations
import plotly.graph_objects as go 

In [ ]:
from functions import concatenate_eurostat_countries, cleaning_eurostat_data_for_viz, clean_countries_numbeo, clean_cities
eurostat_df = concatenate_eurostat_countries()
eurostat_df

In [ ]:
cities = clean_cities()

In [ ]:
countries = clean_countries_numbeo()
countries

In [ ]:
income_tidy, housing_tidy, rental_tidy, income_df, housing_df, rental_df = cleaning_eurostat_data_for_viz(eurostat_df)
income_df

In [ ]:
# Concatenate the DataFrames into one long DataFrame
combined_data = pd.concat([housing_df, rental_df], axis=0)
df_portugal = combined_data[combined_data['Country'] == 'PT']
df_portugal = df_portugal.reset_index().rename(columns={'index': 'Type'})
df_portugal_melted = df_portugal.melt(id_vars=['Type', 'Country'], var_name='Year', value_name='Value')
df_portugal_melted

In [ ]:
fig = px.line(
    df_portugal_melted,
    x='Year',
    y='Value',
    color='Type',  # Different lines for Income and Rental
    markers=True,
    title="Housing vs Rental in Portugal (2019-2023)"
)

fig.show()

In [ ]:
portugal_income = income_tidy[income_tidy["Country"] == "PT"]
fig = px.line(
    portugal_income,
    x='Year',          # x-axis: Year
    y='Income',        # y-axis: Income
    markers=True,      # Add markers at each data point
    title="Average income in Portugal (2019-2023)"  # Chart title
)

# Show the plot
fig.show()

In [ ]:
cities_one_bed_apt = cities[cities['Type'] == "1 bed apartment (rent)"]
cities_one_bed_apt_melted = cities_one_bed_apt.melt(id_vars=["Type", "City"], var_name="Year", value_name="Value")
fig = px.line(
    cities_one_bed_apt_melted, 
    x="Year", 
    y="Value",
    color="City",       
    line_group="City",  
    facet_col="Type",      
    markers=True 
)

fig.show()

In [ ]:
cities_salaries = cities[cities['Type'] == "Av salary (after tax)"]
cities_salaries_melted = cities_salaries.melt(id_vars=["Type", "City"], var_name="Year", value_name="Value")
fig = px.line(
    cities_salaries_melted, 
    x="Year", 
    y="Value",
    color="City",       
    line_group="City",  
    facet_col="Type",      
    markers=True,  # Added comma here
    title="Average salaries per month in Lisbon, Berlin and Paris (2019-2024)",  # Title of the chart
    labels={"Value": "Amount (€)", "Year": "Year"}  # Axis labels
)

fig.show()

In [ ]:
three_bed_apt_rent = cities[cities['Type'] == "3 bed apartment (rent)"]
three_bed_apt_rent = three_bed_apt_rent.melt(id_vars=["Type", "City"], var_name="Year", value_name="Value")
fig = px.line(
    three_bed_apt_rent, 
    x="Year", 
    y="Value",
    color="City",       
    line_group="City",  
    facet_col="Type",      
    markers=True, 
     title="Average rent for 3bedroom apartment per month in Lisbon, Berlin and Paris (2019-2024)",  # Title of the chart
    labels={"Value": "Amount (€)", "Year": "Year"} )

fig.show()

In [ ]:
df_lisbon = cities[cities["City"] == "Lisbon"]
df_portugal = countries[countries["Country"] == "PT"]
display(cities)
display(countries)

In [ ]:
national_average = pd.concat([df_lisbon, df_portugal], ignore_index=True)    
national_average.rename(columns={"City": "City/Country"}, inplace=True) 
national_average.drop(columns=["Country"], inplace=True)
national_average.loc[[4,5,6,7,8], "City/Country"] = "Portugal"
national_average

In [ ]:
salary_data = national_average[
    (national_average['Type'].isin(['Av salary (after tax)', 'Min wage (after tax)'])) &
    (national_average['City/Country'].isin(['Portugal', 'Lisbon']))]

salary_data_melted = salary_data.melt(id_vars=["Type", "City/Country"], var_name="Year", value_name="Value")
fig = px.line(
    salary_data_melted,
    x="Year",
    y="Value",
    color="Type",
    line_dash="City/Country",   # Different lines for Lisbon and Portugal
    markers=True,
    title="Average Salary and Minimum Wage in Lisbon with National Average (2019-2024)",
    labels={"Value": "Amount (€)", "Year": "Year", "Type": "Type of Income"}
)

fig.show()